<a href="https://colab.research.google.com/github/unachankz/goto_data_science/blob/master/notebook/faiss_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# 乱数の固定
np.random.seed(1234)             # make reproducible

# データベース（探索先）の作成（64次元10万エントリ）
d = 64                           # dimension
nb = 100000                      # database size
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

In [2]:
# クエリベクトル群の作成(64次元1万クエリ)
nq = 10000                       # nb of queries
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.


In [3]:
# ライブラリのインポート
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-cpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [4]:
d

64

In [5]:
nlist = 100
m = 8
k = 4

quantizer = faiss.IndexFlatL2(d)  # this remains the same

In [6]:
quantizer

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f4480870450> >

In [7]:
# モデルの作成？
# 引数はそれぞれ量子化対象のインデックス、データの次元数、転置インデックスのリストサイズ、サブベクトル数、サブベクトルの量子化ビット数を表す
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                  # 8 specifies that each sub-vector is encoded as 8 bits


In [8]:
# ぽかチェック（とりあえずデータベースの最初の5エントリーに近いベクトルをk個探す）
D, I = index.search(xb[:5], k) # sanity check
# Iは検索したベクトルのインデックス、Dは検索したベクトルのユークリッド距離
# もちろんIの各ベクトルの一要素目はベクトル自身が入る
print(I)
print(D)

[[-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]]
[[3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]
 [3.4028235e+38 3.4028235e+38 3.4028235e+38 3.4028235e+38]]


In [9]:
# 実際に今回のクエリを検索にかける
index.nprobe = 10              # make comparable with experiment above
# D, I = index.search(xq, k)     # search

# print(len(D))
# print(len(I))

In [ ]:
D, I = index.search(xq, k)     # search

print(len(D))
print(len(I))